# Data clean - preparation - MyImmo

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Téléchargement des fichiers csv

In [2]:
import urllib.request
def download_files_etalab():
  years_files = ['2014', '2015', '2016', '2017', '2018', '2019', '2020']
  for year in years_files:
    url_files = "https://cadastre.data.gouv.fr/data/etalab-dvf/latest/csv/" + year + "/full.csv.gz"
    name_file = "full-" + year +".csv.gz"
    path_file = '/home/' + name_file
    urllib.request.urlretrieve(url_files, path_file)

In [3]:
download_files_etalab()

### Traitement sur les données

In [4]:
import pandas as pd
import os

Import des fichiers

In [5]:
def import_files_in_dict(year):
  path_file = '/home/full-' + year + '.csv.gz'
  df = pd.read_csv(path_file, compression='gzip')
  return df

Clean data

In [6]:
def clean_data(df):
  df = df[ (df['nature_mutation'] == 'Vente') | (df['nature_mutation'] == "Vente en l'état futur d'achèvement")]
  df = df[ (df['valeur_fonciere'].notna()) & (df['valeur_fonciere'] != 0)]
  df = df[ (df['surface_reelle_bati'].notna()) & (df['surface_reelle_bati'] != 0)]
  df = df[ (df['longitude'].notna())]
  df = df[ (df['latitude'].notna()) ]
  df = df[ (df['nombre_lots'] == 0) ]
  df = df[ (df['code_departement'].notna()) ]
  return df

Traitement sur les données


In [7]:
file_regions = "/content/drive/MyDrive/departments.csv"
regions = pd.read_csv(file_regions)

In [8]:
output_data = ['id_mutation', 'date_mutation', 'nature_mutation',
               'valeur_fonciere', 'code_postal', 'code_commune', 'code_region',
               'code_departement', 'code_type_local', 'type_local',
               'surface_reelle_bati', 'nombre_pieces_principales', 'latitude', 'longitude']
years_files = ['2015', '2016', '2017', '2018', '2019', '2020']

df_immo = [] 
for year in years_files:
  df = import_files_in_dict(year)
  df["code_departement"] = df["code_departement"].astype(str)
  df["code_departement"] = df["code_departement"].apply(lambda x: '0' + x if len(x)==1 else x)
  df = pd.merge(df, regions, how='left', left_on='code_departement', right_on='code')
  df["code_region"] = df["region_code"]
  df_clean = clean_data(df)
  df_clean = df_clean[output_data]
  df_clean["prix_metre_carre"] = (df_clean['valeur_fonciere']/df_clean['surface_reelle_bati']).apply(lambda x: round(x))
  df_clean = df_clean.sort_values(['code_region', 'latitude','longitude'])
  if len(df_immo) == 0:
    df_immo = df_clean
  else:
    df_immo = pd.concat([df_immo, df_clean])
df_immo

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (10,12,14,16,17,18,20,22,24,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (10,12,14,16,17,18,20,22,24,26,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (10,12,14,16,17,18,20,22,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (10,12,14,16,17,18,20,22,26,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.7/dist-pa

,id_mutation,date_mutation,nature_mutation,valeur_fonciere,code_postal,code_commune,code_region,code_departement,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,latitude,longitude,prix_metre_carre
2663480,2015-1131554,2015-06-25,Vente,1860.0,97136.0,97130,01,971,1.0,Maison,90.0,4.0,15.848188,-61.644636,21
2663556,2015-1131605,2015-07-08,Vente,1570.0,97136.0,97130,01,971,1.0,Maison,98.0,4.0,15.848572,-61.644027,16
2663557,2015-1131605,2015-07-08,Vente,1570.0,97136.0,97130,01,971,2.0,Appartement,37.0,2.0,15.848572,-61.644027,42
2664470,2015-1132114,2015-12-23,Vente,1450.0,97136.0,97130,01,971,1.0,Maison,148.0,5.0,15.848833,-61.643867,10
2663237,2015-1131405,2015-04-09,Vente,720000.0,97137.0,97131,01,971,1.0,Maison,88.0,3.0,15.862501,-61.597274,8182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558874,2020-227595,2020-12-31,Vente,375000.0,20600.0,2B120,94,2B,1.0,Maison,138.0,5.0,42.649121,9.443310,2717
558800,2020-227568,2020-11-16,Vente,180000.0,20200.0,2B353,94,2B,1.0,Maison,104.0,3.0,42.714854,9.427883,1731
558817,2020-227575,2020-12-17,Vente,123500.0,20200.0,2B305,94,2B,1.0,Maison,33.0,1.0,42.733560,9.460780,3742
558794,2020-227564,2020-10-14,Vente,265000.0,20233.0,2B224,94,2B,1.0,Maison,82.0,4.0,42.840266,9.456825,3232


Sauvegarde du fichier clean et preparer

In [9]:
save_file_path = "/content/drive/MyDrive/dataset-myimmo.csv"
df_immo.to_csv(path_or_buf=save_file_path,index=False,header=True)

Modele BallTree - prix moyen du voisinage

In [10]:
pd.pivot_table(df_immo, values='valeur_fonciere', index=['type_local'],columns=['nature_mutation'], aggfunc=lambda x: len(x))

nature_mutation,Vente,Vente en l'état futur d'achèvement
type_local,,
Appartement,654187.0,1452.0
Dépendance,107.0,7.0
Local industriel. commercial ou assimilé,299832.0,818.0
Maison,3361044.0,4076.0


On creer un ensemble de modele pour les appartements et les maison 

In [31]:
# Appartement
df_immo_appart = df_immo[ (df_immo["nature_mutation"]=="Vente") & (df_immo["type_local"]=="Appartement") ]
# Maison
df_immo_maison = df_immo[ (df_immo["nature_mutation"]=="Vente") & (df_immo["type_local"]=="Maison") ]
df_immo_maison

,id_mutation,date_mutation,nature_mutation,valeur_fonciere,code_postal,code_commune,code_region,code_departement,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,latitude,longitude,prix_metre_carre
2663480,2015-1131554,2015-06-25,Vente,1860.00,97136.0,97130,01,971,1.0,Maison,90.0,4.0,15.848188,-61.644636,21
2663556,2015-1131605,2015-07-08,Vente,1570.00,97136.0,97130,01,971,1.0,Maison,98.0,4.0,15.848572,-61.644027,16
2664470,2015-1132114,2015-12-23,Vente,1450.00,97136.0,97130,01,971,1.0,Maison,148.0,5.0,15.848833,-61.643867,10
2663237,2015-1131405,2015-04-09,Vente,720000.00,97137.0,97131,01,971,1.0,Maison,88.0,3.0,15.862501,-61.597274,8182
2664469,2015-1132113,2015-12-30,Vente,10671.43,97137.0,97131,01,971,1.0,Maison,38.0,3.0,15.862801,-61.586874,281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558874,2020-227595,2020-12-31,Vente,375000.00,20600.0,2B120,94,2B,1.0,Maison,138.0,5.0,42.649121,9.443310,2717
558800,2020-227568,2020-11-16,Vente,180000.00,20200.0,2B353,94,2B,1.0,Maison,104.0,3.0,42.714854,9.427883,1731
558817,2020-227575,2020-12-17,Vente,123500.00,20200.0,2B305,94,2B,1.0,Maison,33.0,1.0,42.733560,9.460780,3742
558794,2020-227564,2020-10-14,Vente,265000.00,20233.0,2B224,94,2B,1.0,Maison,82.0,4.0,42.840266,9.456825,3232


Modele BallTree pour les appartements - 1 modéle par region

In [28]:
from sklearn.neighbors import BallTree
import numpy as np
import time

In [23]:
def save_obj(obj,name):
            with open( name + '.pkl', 'wb') as f:
                pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [32]:
df_immo_appart['distance_moyenne']=np.zeros(len(df_immo_appart))
df_immo_appart['index_voisins']=np.zeros(len(df_immo_appart))
models = {}
regions = df_immo_appart.code_region.unique()
for k in range(len(regions)):
    name = 'appart_' + regions[k]
    data = df_immo_appart[df_immo_appart.code_region==regions[k]]
    data = data.reset_index(drop=True)
    models[k] = BallTree(data[['latitude', 'longitude']].values, leaf_size=2, metric='haversine')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [33]:
for k in range(len(regions)):
    start = time.time()
    data = df_immo_appart[df_immo_appart.code_region==regions[k]]
    data = data.reset_index(drop=True)
    dist, indices = models[k].query(data[['latitude','longitude']].values,k=10)
    data['distance_moyenne'] = np.mean(dist[:,1:]*6341,1)
    a = pd.DataFrame()
    a['prix_metre_carre'] = np.zeros(len(data))
    for i in range(1,10):
        a += pd.DataFrame(data.iloc[indices[:,i],:]['prix_metre_carre']).reset_index(drop=True)
    a=a/10
    data['prix_moyen_cartier'] = a.values
    stop = time.time()
    print(stop-start)
    data.to_csv("/content/drive/MyDrive/data_pa/appart_region_" + regions[k] +".csv", index=False, header=True)

0.28316569328308105
0.7202389240264893
0.49651455879211426
0.8082408905029297
143.52995109558105
3.7611513137817383
3.706237316131592
5.784782409667969
8.545553922653198
7.297018766403198
4.167701959609985
5.06593656539917
13.094555139541626
10.877479314804077
28.122822761535645
9.212506294250488
0.11952781677246094
